In [ ]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import time

from constants import DATA_DIR

In [ ]:
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [ ]:
url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}

In [ ]:
response = requests.post(url=url, headers=headers, data=data)

In [ ]:
response.json()

In [ ]:
access_token = response.json()["access_token"]

In [ ]:
def playlist_query(playlist_id, access_token=access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    try:
        response = requests.get(f'https://api.spotify.com/v1/playlists/{playlist_id}', headers=headers)
    
        if response.status_code == 200:
            print('API request succeeded!')
            response_data = response.json()
            return response_data
        else:
            print(f'Request failed. Expected 200, got {response.status_code}: {response.text}')
            return None
    except requests.exceptions.RequestException as e:
        print(f'An error occurred during the request: {e}')
        return None

In [ ]:
#response['tracks']['items'][0]['track']['artists'][0]['name']
#response['tracks']['items'][0]['track']['artists'][0]['id']



In [ ]:
df = pd.read_csv(f'{DATA_DIR}/playlist.txt', header=None)
playlist_list = df.values.tolist()[0]

artist_df = pd.DataFrame(columns=['name','artist_id'])

In [ ]:
type(artist_df)

In [ ]:
for playlist in playlist_list:
    time.sleep(10)
    response = playlist_query(playlist)
    for track in response['tracks']['items']:
        name = track['track']['artists'][0]['name']
        artist_id = track['track']['artists'][0]['id']

        is_unique = artist_df.loc[(artist_df['name'] == name) & (artist_df['artist_id'] == artist_id)].empty

        if is_unique:
            new_row = pd.Series({'name': name, 'artist_id': artist_id})
            artist_df = pd.concat([artist_df, pd.DataFrame([new_row])], ignore_index=True)
        


In [ ]:
artist_df.to_csv(f'{DATA_DIR}/artists.csv', index=False)